<a href="https://colab.research.google.com/github/Aryan-401/GoogleColabProjects/blob/main/WhisperTiny_Finetune_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
https://github.com/libindic/indic-trans
"""
!git clone https://github.com/libindic/indic-trans.git

Cloning into 'indic-trans'...
remote: Enumerating objects: 2225, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 2225 (delta 8), reused 10 (delta 3), pack-reused 2206
Receiving objects: 100% (2225/2225), 516.51 MiB | 22.89 MiB/s, done.
Resolving deltas: 100% (1102/1102), done.
Updating files: 100% (719/719), done.


In [ ]:
!pip install -r indic-trans/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install indic-trans/.

Processing ./indic-trans
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for indictrans: filename=indictrans-1.2.3-cp310-cp310-linux_x86_64.whl size=337822735 sha256=d1389760c76720f281cf51e1afa1518bd5510cd7b0382f0dfe6d29eb9ba83298
  Stored in directory: /root/.cache/pip/wheels/3e/c9/43/39c5aaa9a570043089bac219a37343294fa5b47f1350dea53a
Successfully built indictrans


congress party adhyaksh sonia gandhi, tamilnadu kii mukhyamantri jayalalita or reserve bank ke governor raghuram rajan ke bich ek samaantahai. ye sabhi alag-alag kaarnon se bhartiya janata party ke rajyasabha saansad subramanyam swami ke nishane par hai. unke jayalalita or sonia gandhi ke peeche padane kaa kaaran kathith bhrashtachar he.


In [ ]:
!pip install transformers[torch] datasets[audio] evaluate jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("google/fleurs", 'hi_in', split='train+test')

In [ ]:
print(dataset)

Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 2538
})


In [ ]:
dataset = dataset.select_columns(["audio", "transcription"])

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-tiny", language="hindi", task="transcribe"
)

In [ ]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [ ]:
dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 2538
})

In [ ]:
from indictrans import Transliterator

def convert_hindi(example):
    audio = example["audio"]
    trn = Transliterator(source='hin', target='eng', build_lookup=True)
    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=trn.transform(example['transcription']),
    )
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    return example


In [ ]:
dataset = dataset.map(
    convert_hindi, remove_columns=dataset.column_names ,num_proc=1, batch_size=100
)

Map:   0%|          | 0/2538 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 2538
})

In [ ]:
dataset = dataset.train_test_split(0.1)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 2284
    })
    test: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 254
    })
})

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union



class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __init__(self, processor):
        self.processor = processor

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="hindi", task="transcribe", use_cache=True
)

In [ ]:
from transformers import Seq2SeqTrainingArguments

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-finetune-hindi-fleurs",  # name on the HF Hub
    per_device_train_batch_size=32,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [ ]:
kwargs = {
     "dataset_tags": "google/fleurs",
    "finetuned_from": "openai/whisper-tiny",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer Ortho,Wer
100,1.811200,1.727381,0.632327,0.625800
200,1.038700,1.119369,0.512999,0.507202
300,0.767100,0.967135,0.466494,0.461268
400,0.528300,0.884007,0.449378,0.443982
500,0.445800,0.831486,0.431293,0.426216


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=500, training_loss=1.2182455615997314, metrics={'train_runtime': 3166.8649, 'train_samples_per_second': 5.052, 'train_steps_per_second': 0.158, 'total_flos': 3.909478957056e+17, 'train_loss': 1.2182455615997314, 'epoch': 6.94})

In [ ]:
trainer.push_to_hub(**kwargs)

'https://huggingface.co/Aryan-401/whisper-tiny-finetune-hindi-fleurs/tree/main/'